In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any, Literal
from tqdm import tqdm
import time

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [5]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df


In [7]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [8]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [9]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [10]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


Правило обновления в функции __get_pagerank
$$ \vec{x} \leftarrow a P \vec{x} + b \begin{bmatrix} p^{(u)} \vec{x} \\ p^{(u)} \vec{x} \\ \vdots \\ p^{(u)} \vec{x} \end{bmatrix} + c \begin{bmatrix} \frac{1}{n} \cdots \frac{1}{n} \end{bmatrix}^{T}$$

In [11]:

MappingIntToObject = Mapping[int, Tuple[Any, Literal['source', 'target']]]
        
class PageRank():
    """ Код класса является переработкой для конкретной задачи кода из
    https://github.com/danieljunhee/Tutorial-on-Personalized-PageRank/blob/master/Personalized_PageRank_Tutorial.ipynb
    """
    
    def __init__(self, data:pd.DataFrame, source: str, target:str, edge_attr:str,
                 personalize_by: Any, a: float = 0.8, b: float = 0.15, c: float=0.05,
                 max_iter: int = 100) -> None:
        # Decide values of a,b,c. For this tutorial, let's use a = 0.8, b = 0.15, c = 0.05.
        self.a = a
        self.b = b
        self.c = c
        self.target = target
        self.source = source
        self.max_iter = max_iter
        
        # Запоминаем матрицу переходов, вероятности переходов для персонализируемой вершины,
        # и отображение целых чисел (индексов) в исходные объекты
        (self.__transition_matrix, self.__user_person_vec,
         self.__N_to_obj) = self.__create_transition_matrix_from_pd(data, source,
                                                             target, edge_attr, personalize_by)
         
        
    def __create_transition_matrix_from_pd(self, data: pd.DataFrame,
               source: str, target: str, edge_attr: str,
               personalization: Any) -> Tuple[sparse.csr_matrix,
                                              sparse.csr_matrix,
                                              MappingIntToObject]:
        """ Вспомогательная функция для постороения матрицы переходов

        Args:
            data (pd.DataFrame): Данные представляющие список ребер
            source (str): название столбца с исходными вершинами
            target (str): название столбца с конечными вершинами
            edge_attr (str): название столбца с длиной ребра
            personalization (Any): персонализуруемый объект из столбца source

        Returns:
            Tuple[sparse.csr_matrix, sparse.csr_matrix, MappingIntToObject]: 
            матрица переходов, вероятности перехода для персонализируемого объекта,
            отображение множества целых чисел в множество исходных объектов из data
        """
    
        obj_source_to_N = {}
        obj_target_to_N = {}
        N_to_obj = {}
        
        source_uniq = set(data[source])
        for index, obj in zip(range(len(source_uniq)), source_uniq):
            obj_source_to_N[obj] = index
            N_to_obj[index] = (obj, 'source')
            
        
        target_uniq = set(data[target])
        for index, obj in zip(range(len(source_uniq), len(source_uniq) + len(target_uniq)), target_uniq):
            obj_target_to_N[obj] = index
            N_to_obj[index] = (obj, 'target')
        
        count_top = len(N_to_obj)
        transition_matrix = sparse.lil_matrix((count_top, count_top))
        
        # Функция для отображения значений согласно словарю
        def map_source_to_N(value):
            return obj_source_to_N[value]

        def map_target_to_N(value):
            return obj_target_to_N[value]

        # Создаем векторизованную версию функций
        vectorized_source_to_N = np.vectorize(map_source_to_N)
        vectorized_target_to_N = np.vectorize(map_target_to_N)

        # Применяем векторизованные функции к массивам
        start_nodes = vectorized_source_to_N(data[source].to_numpy())  # Начальные вершины ребер
        end_nodes = vectorized_target_to_N(data[target].to_numpy())    # Конечные вершины ребер
        weights = np.array(data[edge_attr])
        
        # Заполнение матрицы значениями из массивов
        transition_matrix[start_nodes, end_nodes] = weights

        # Если граф неориентированный, добавьте обратные ребра
        transition_matrix[end_nodes, start_nodes] = weights
        
        # Получаем суммы элементов каждой строки в виде numpy массива
        
        transition_matrix_csr = sparse.csr_matrix(transition_matrix)
        del transition_matrix
        
        row_sums = transition_matrix_csr.sum(axis=1)
        transition_matrix_csr /= row_sums
        
        user_person_vec = np.zeros(transition_matrix_csr.shape[1])
        #user_person_vec[obj_source_to_N[personalization]] = 1
        pers_data = data.loc[data[source] == personalization]
        for ind, row in pers_data.iterrows():
            user_person_vec[obj_target_to_N[row[target]]] = row[edge_attr]
        user_person_vec /= user_person_vec.sum()

        # transition_matrix_csr.getrow(obj_source_to_N[personalization]).transpose()
        return (transition_matrix_csr,
                transition_matrix_csr.getrow(obj_source_to_N[personalization]).transpose(),
                N_to_obj)
    
    def get_pagerank(self) -> pd.DataFrame:
        """ Целевая функция, которая возращает pagerank

        Returns:
            pd.DataFrame: Возвращает датафрейм с двумя колонками (source, pagerank)
        """
        
        pagerank = self.__get_pagerank(self.__transition_matrix,
                                       self.__user_person_vec, self.max_iter)
         
         
        target_column = []
        pagerank_column = []
        for ind, el_pagerank in enumerate(pagerank):
            if self.__N_to_obj[ind][1] == 'target':
                target_column.append(self.__N_to_obj[ind][0])
                pagerank_column.append(el_pagerank)

        return pd.DataFrame({self.target: target_column , 'pagerank': pagerank_column})
    
    def __get_pagerank(self, transition_matrix: sparse.csr_matrix,
                       user_person_vec: sparse.csr_matrix, max_iter: int = 200) -> np.ndarray:
        """ Метод возращает pagerank. Останавливается, есть достигнута нужная точновть или превышен лимит итераций.
        

        Args:
            transition_matrix (sparse.csr_matrix): матрица переходов
            user_person_vec (sparse.csr_matrix): вероятности переходов для персонализируемого объекта
            max_iter (int, optional): макисиальное кол-во итераций. Defaults to 200.

        Returns:
            np.ndarray: pagerank для каждой вершины (вершины натуральные числа)
        """
        
        # Initialize PageRank vector: height x 1 vector of all 1/height 's
        # We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
        # is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
        
        transition_matrix_T_mult_a = self.a * transition_matrix.transpose()
        user_person_vec_mult_b = self.b* user_person_vec
        height = transition_matrix.shape[0]
        user_pagerank_vec = sparse.csr_matrix(np.ones((height,1)) / height)
        sup_vec = self.c*sparse.csr_matrix((1/height)*np.ones((height,1)))

        # Инициализация прогресс-бара
        progress_bar = tqdm(total=max_iter)
        # Update the PageRank vector until convergence! Our convergence criterion is to see whether
        # the magnitude of the difference of the PageRank vector after an update is 0
        for i in range(max_iter):
            # (deep) copy the current PageRank vector to compare with the updated vector
            old_user_pagerank_vec = user_pagerank_vec.copy()
            
            # Update!
            user_pagerank_vec = transition_matrix_T_mult_a.dot(user_pagerank_vec) \
                                + user_person_vec_mult_b.multiply(user_pagerank_vec) \
                                + sup_vec 
                                
            if i % 10 == 0:
                user_pagerank_vec /= user_pagerank_vec.sum()
            
            magnitude = norm(user_pagerank_vec-old_user_pagerank_vec)
            progress_bar.set_description("magnitude: {:.6f}".format(magnitude))
            progress_bar.update()
            
            if magnitude == 0:
                break
        progress_bar.close()
        return np.concatenate(user_pagerank_vec.toarray())
    
    def set_params(self, a: float = None, b: float = None, c: float = None,
                   max_iter: int = None) -> None:
        self.a = a if a is not None else self.a
        self.b = b if b is not None else self.b
        self.c = c if c is not None else self.c
        self.max_iter = max_iter if max_iter is not None else self.max_iter
    
  
data_test = pd.DataFrame({
    'source': ['A', 'B', 'B', 'B'],
    'target': ['X', 'X', 'Z', 'C'],
    'edge_attr': [4, 4, 5, 3]
})  
    
pg = PageRank(data_test, 'source', 'target', 'edge_attr', 'A')
pg.get_pagerank()

magnitude: 0.013561: 100%|██████████| 100/100 [00:00<00:00, 261.97it/s]


,target,pagerank
0,C,0.043405
1,Z,0.065675
2,X,0.130953


In [12]:
pg = PageRank(data, 'userId', 'movieId', 'rating', 1, max_iter=100)

In [13]:
# b отвечает за персонализацию за персонализацию
pg.set_params(a=0.8, b=0.15, c=0.05)

In [14]:
prediction_movie_for_user1 = pg.get_pagerank()

magnitude: 0.000151: 100%|██████████| 100/100 [01:53<00:00,  1.13s/it]


In [15]:
print(prediction_movie_for_user1.sort_values(by='pagerank', ascending=False).head())


     movieId  pagerank
317      318  0.000767
295      296  0.000750
354      356  0.000708
589      593  0.000704
259      260  0.000579


In [16]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['movieId'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

26657


,movieId,pagerank
0,131072,3.168565e-07
1,1,5.093424e-04
2,2,1.720720e-04
3,3,1.165603e-04
4,4,1.949024e-05


In [17]:
# Топ фильмов по мнению pagerank
pred = prediction_movie_for_user1_without_famous.sort_values(by='pagerank', ascending=False).head(len(test1))
print(len(pred))
pred.head()

88


,movieId,pagerank
317,318,0.000767
295,296,0.000750
354,356,0.000708
259,260,0.000579
478,480,0.000575


In [18]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [19]:
print(len(test1['movieId']))
test1['movieId'].head()

88


156    7045
145    6242
101    3081
127    4911
141    5898
Name: movieId, dtype: int64

In [20]:
print(len(pred['movieId']))
pred['movieId'].head()

88


317    318
295    296
354    356
259    260
478    480
Name: movieId, dtype: int64

In [21]:
print(accuracy_without_order(test1['movieId'], pred['movieId']))

0.25


In [22]:
movie_df = pd.read_csv('res/movie.csv')

In [23]:
prediction_moviename = pd.merge(movie_df, pred, on='movieId', how='inner')
prediction_moviename.sort_values(by='pagerank', ascending=False).head()

,movieId,title,genres,pagerank
21,318,"Shawshank Redemption, The (1994)",Crime|Drama,0.000767
19,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0.000750
26,356,Forrest Gump (1994),Comedy|Drama|Romance|War,0.000708
16,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,0.000579
33,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,0.000575


In [24]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

88


,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [25]:
# Фильмы которые были угаданны
test_moviename[test_moviename['movieId'].isin(prediction_moviename['movieId'])]

,movieId,title,genres,userId,rating,timestamp
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1,3.5,2005-04-02 23:33:39
2,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,3.5,2005-04-02 23:29:40
4,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1,4.0,2005-04-02 23:33:46
5,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,1,4.0,2005-04-02 23:31:43
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1,4.0,2005-04-02 23:32:47
7,318,"Shawshank Redemption, The (1994)",Crime|Drama,1,4.0,2005-04-02 23:33:18
8,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1,4.0,2005-04-02 23:30:03
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1,3.5,2005-04-02 23:45:57
12,1036,Die Hard (1988),Action|Crime|Thriller,1,4.0,2005-04-02 23:44:40
15,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller,1,3.5,2005-04-02 23:31:09


In [26]:
from typing import Any, List

class O:
    
    def __init__(self, delta: Any) -> None:
        self.delta = delta
        
    def __call__(self, true_score_value: Any) -> int:
        return int(true_score_value > self.delta)
    
class T():
    def __init__(self, delta: Any) -> None:
        self.delta = delta
        
    def __call__(self, pred_value: Any) -> int:
        return int(pred_value > self.delta)

class I():
    
    def __init__(self, pagerankT:T, valuesO: O) -> None:
        self.pagerankT = pagerankT
        self.valuesO = valuesO
        
    def __call__(self, pred_p: Any, true_p: Any,
                 pred_q: Any, true_q: Any) -> int:
        if pred_p >= pred_q and \
            self.valuesO(true_p) < self.valuesO(true_q):
            return 1
        if pred_p <= pred_q and \
            self.valuesO(true_p) > self.valuesO(true_q):
            return 1
        return 0
    
    
class Pairord():
    
    def __init__(self, i: I) -> None:
        self.i = i
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        metric = len(pred_pagerank) * len(true_score)
        for p in range(len(pred_pagerank)):
            for q in range(len(true_score)):
                metric -= self.i(pred_pagerank[p], true_score[q],
                                        pred_pagerank[q], true_score[p])
            
        metric /= (len(pred_pagerank) * len(true_score))
        
        return metric
    
class Prec():
    
    def __init__(self, o: O, t: T) -> None:
        self.o = o
        self.t = t       
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        numerator = 0
        denominator = 0
        for p in range(len(pred_pagerank)):
            numerator += self.t(pred_pagerank[p]) * self.o(true_score[p])
            denominator += self.t(pred_pagerank[p])
        
        return numerator / denominator
    
    
class Rec():
    
    def __init__(self, o: O, t: T) -> None:
        self.o = o
        self.t = t       
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        numerator = 0
        denominator = 0
        for p in range(len(pred_pagerank)):
            numerator += self.t(pred_pagerank[p]) * self.o(true_score[p])
            denominator += self.o(true_score[p])
        
        return numerator / denominator
    
class F1():
    
    def __init__(self, prec: Prec, rec: Rec) -> None:
        self.prec = prec
        self.rec = rec
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        p = self.prec(pred_pagerank, true_score)
        r = self.rec(pred_pagerank, true_score)
            
        return 2*p*r/(p + r)
        
    
        

In [27]:
for_score = prediction_movie_for_user1.merge(test1, how='inner', on='movieId')
o = O(test1['rating'].mean())
t = T(prediction_movie_for_user1['pagerank'].mean())
i = I(t, o)
pairroid = Pairord(i)
print(pairroid(for_score['pagerank'], for_score['rating']))

0.6701962809917356


In [28]:
prec = Prec(o, t)
print(prec(for_score['pagerank'], for_score['rating']))

0.525


In [29]:
rec = Rec(o, t)
print(rec(for_score['pagerank'], for_score['rating']))

0.9333333333333333


In [30]:
f1 = F1(prec, rec)
print(f1(for_score['pagerank'], for_score['rating']))

0.672


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
pred = np.array(for_score['pagerank'])
true = np.array(for_score['rating'])
pred_map = np.vectorize(lambda x: t(x))
true_map = np.vectorize(lambda x: o(x))

pred_trasformed = pred_map(pred)
true_trasformed = true_map(true)

In [32]:
# Вычисляем метрики
accuracy = accuracy_score(true_trasformed, pred_trasformed)
precision = precision_score(true_trasformed, pred_trasformed)
recall = recall_score(true_trasformed, pred_trasformed)
f1 = f1_score(true_trasformed, pred_trasformed)

# Выводим результаты
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5340909090909091
Precision: 0.525
Recall: 0.9333333333333333
F1 Score: 0.672
